In [1]:
import numpy as np
import pandas as pd
import sqlite3
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn import manifold
#from cuml.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm
from nltk import word_tokenize


In [66]:
data = pd.read_csv('data_featuring_all.txt', sep='|')#, nrows=1000)

In [2]:
wordvectors_file_vec = 'embeddings-s-model.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [3]:
data.head(2)

,titulo,cuerpo,fecha_publicacion,diario,url,page,year,month,week,id,long_cuerpo,pre_clean_text,clean_text,municipios,departamentos,text_for_embedding,category_bl,no_embedding_words,centroide
0,Cerca de ocho toneladas de coca del Clan del G...,En una operación de la Policía Nacional y la A...,2020-07-01 17:43:00,diariodelcauca,https://diariodelcauca.com.co/noticias/judicia...,3,2020,7,26,aHR0cHM6Ly9kaWFyaW9kZWxjYXVjYS5jb20uY28vbm90aW...,225,cerca ocho toneladas coca clan golfo cayeron o...,cerc ocho tonel clan golf cayeron operacion co...,colon|cartagena,putumayo|narino|bolivar,cerca ocho toneladas clan golfo cayeron operac...,2,{'caolin'},"[-0.4902389645576477, 0.25123387575149536, -0...."
1,"Capturado alias ‘Arturo’, segundo cabecilla de...",Sería el encargado de organizar esta estructur...,2020-07-01 18:40:00,diariodelcauca,https://diariodelcauca.com.co/noticias/judicia...,3,2020,7,26,aHR0cHM6Ly9kaWFyaW9kZWxjYXVjYS5jb20uY28vbm90aW...,420,capturado alias arturo segundo cabecilla disid...,captur ali cabecill disident tolim vide encarg...,marquetalia,tolima,capturado alias cabecilla disidencias tolima v...,1,"{'extorsionando', 'disidencias', 'planadas', '...","[-0.5124386548995972, 0.2755158245563507, -0.2..."


In [4]:
#Hecho contra la población civil
categoria0 = [
                "homicidio",
                 "masacre", 
                 "secuestro",
                 "tortura",
                 "desplazamiento",
                 "forzado",
                 "confinamiento",
                 "accidente",
                 "mina",
#                 "antipersona",
                 "atentado",
                 "reclutamiento",
                 "violencia",
                 "sexual",
                 "ataque",
                 "indiscriminado",
                 "bienes",
                 "civiles"
]

#acciones armadas
categoria1 = [
                "combate",
                "enfrentamiento",
                "hostigamiento",
                "emboscada",
                "acto",
                "terrorismo",
                "ataque",
                "instalación",
                "militar",
                "policía" ,
                "restricción",
                "movilidad", 
                "retenes",
                "ilegal", 
                "paro",
                "armado",
                "indiscriminado",
                "incursión",
                "población"
]

# Acciones institucionales
categoria2 = [
                "captura",
                "desactivación",
                "artefacto",
                "explosivo",
#                "antinarcotico",
                "incautación",
                "rescate",
                "secuestrado",
                "liberación",
                "misión",
                "humanitaria"
]

#Otro tipo de evento
categoria3 = [
                "pronunciamiento",
                "organismo",
                "internacional",
                "nacional",
                "conflicto",
                "derecho",
                "humano",
                "ddhh",
                "victimización"
]

In [5]:
def similitud_categorias(palabras):
    sim0 = []
    sim1 = []
    sim2 = []
    sim3 = []
    not_in_we = []
    for w_n in palabras:
        for w in categoria0:
            try:
                sim0.append(wordvectors.similarity(w_n, w))
            except:
                not_in_we.append(w_n)

        for w in categoria1:
            try:
                sim1.append(wordvectors.similarity(w_n, w))
            except:
                not_in_we.append(w_n)

        for w in categoria2:
            try:
                sim2.append(wordvectors.similarity(w_n, w))
            except:
                not_in_we.append(w_n)

        for w in categoria3:
            try:
                sim3.append(wordvectors.similarity(w_n, w))
            except:
                not_in_we.append(w_n)

    sim0 = np.mean(sim0)
    sim1 = np.mean(sim1)
    sim2 = np.mean(sim2)
    sim3 = np.mean(sim3)

    return sim0, sim1, sim2, sim3, not_in_we

In [6]:
def centroide_noticia(palabras, dims=30):
    vectores = []
    for w in palabras:
        try:
            vectores.append(wordvectors[w])
        except:
            pass
        
    vectores = np.matrix(vectores)

    return vectores.median(axis=0).tolist()[0]

In [7]:
s0, s1, s2, s3, l = similitud_categorias(categoria3)
print("Similitud categoria 0:", s0)
print("Similitud categoria 1:", s1)
print("Similitud categoria 2:", s2)
print("Similitud categoria 3:", s3)
print('Palabras no en word embedding:', l)

Similitud categoria 0: 0.47152242
Similitud categoria 1: 0.46367338
Similitud categoria 2: 0.40492046
Similitud categoria 3: 0.5917593
Palabras no en word embedding: []


In [12]:
def encuentra_sim(string):
    
    palabras = word_tokenize(string)
    s0, s1, s2, s3, l = similitud_categorias(palabras)
    
    return max([s0, s1, s2, s3])

In [ ]:
data['sim_cats'] = data['text_for_embedding'].apply(lambda x: encuentra_sim(x))

In [ ]:
data[['sim_cats', 'category_bl']].to_csv('sims_cats.csv', index=False)

In [51]:
data['sim_cats'].head(2)

0    0.405487
1    0.405551
Name: sim_cats, dtype: float64

In [22]:
tsne = pd.read_csv('TSNE_2D.csv')

In [45]:
def graficar_cluster(df, tsne, umbral):
       
    # Input: df, dataframe con las noticias
    #        tnse, dataframe con los vectores
    #        umbral, valor que debe superar la similitud para tener una categoria
    
    # Filtra las noticias cuya similitud supere el umbral
    aux = df.query('sim_cats > @umbral')
    aux['category_bl'] = aux['category_bl'].astype(str)
    indices = aux.index
    
    # Encontrar los vectores en el TNSE
    dfY = tsne.iloc[indices]
    print(dfY)
    
    # Graficar
    plt.figure(figsize=(15,15))
    fig = px.scatter_3d(dfY, x='0', y='1', z='2', color = aux.category_bl)
    fig.show()

In [53]:
graficar_cluster(data, tsne, 0.45)

<ipython-input-45-28d49f4b1f42>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



             0         1          2
269  -7.053811  7.118485  10.776960
384 -14.862395  4.169109  -6.334224
867 -14.200530 -2.616145   5.634569


<Figure size 1080x1080 with 0 Axes>

In [64]:
def graficar_cluster_2D(df, tnse, umbral, colorblind=False):
       
    # Input: df, dataframe con las noticias
    #        tnse, dataframe con los vectores
    #        umbral, valor que debe superar la similitud para tener una categoria
    
    # Filtra las noticias cuya similitud supere el umbral
    aux = df.query('sim_cats > @umbral')
    aux['category_bl'] = aux['category_bl'].astype(str)
    indices = aux.index
    
    # Encontrar los vectores en el TNSE
    dfY = tnse.iloc[indices]
    
    # Graficar
    plt.figure(figsize=(15,15))
    if colorblind:
        fig = px.scatter(dfY, x='0', y='1', color = aux.category_bl, color_discrete_sequence=px.colors.qualitative.Vivid)
    else:
        fig = px.scatter(dfY, x='0', y='1', color = aux.category_bl)
    fig.show()

In [65]:
graficar_cluster_2D(data, tnse, 0.44, True)

<ipython-input-64-3559b9e44566>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<Figure size 1080x1080 with 0 Axes>